# RDF Triples

In [2]:
import rdflib
import xlrd
import pandas
from rdflib import URIRef, Literal, BNode,Namespace

data = { # similar format for how you made your pandas data fram
    "subject": [],
    "predicate": [],
    "object" : []
}
wb = xlrd.open_workbook("ATTWN triples.xlsx")
sheet = wb.sheet_by_index(0)
for i in range(1, sheet.nrows): # skip the first header line
       
    s = sheet.cell_value(i, 0)
    p = sheet.cell_value(i, 1)
    o = sheet.cell_value(i, 2)
    
    data["subject"].append(s)
    data["predicate"].append(p)
    #print(o)
    data["object"].append(o) 
  
data_processed = {
    'subject':[],'predicate':[],'object': []
}
ugly_token = {
    ' ': '_',
    '"': '',
}
# replace all ugly tokens and copy to new data structure
for x in data.keys(): #x is subj,obj,pred
    for item in data[x]:
        if type(item)!=str:
            data_processed[x].append(item)
            continue
        new_token=item       
        for k in ugly_token:
            new_token=new_token.replace(k, ugly_token[k]) 
            
        data_processed[x].append(new_token)            

n = Namespace("http://UCLA_REU_2020.org/ATTWN/")

g = rdflib.Graph()

for i in range(len(data['subject'])):
    s = n[data_processed['subject'][i]]
    p = n[data_processed['predicate'][i]]
    o_data=data_processed['object'][i]
    if type(o_data)==float or type(o_data)==int:
        o_node=Literal(o_data)
    else:
        o_node=n[o_data]
    g.add((s, p, o_node))

#check g
for s, p, o in g:
   print((s, p, o))

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Pills'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/given_by'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_Armstrong'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Character'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/is_character'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_Armstrong'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Vera_Claythorne'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_profession'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/former_governess'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/maid'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_predicate'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/had_killed'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/John_Macarthur'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_lastname'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Macarthur'

# visualization

In [9]:
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib qt
#%matplotlib inline
plt.figure(figsize=(20,20))

edgelabels={}
G = nx.DiGraph()
#plt.clf()
for i in range(len(data['object'])):
    v1 = data['subject'][i]
    v2 = data['object'][i]
    G.add_edge(v1,v2)
    e_lbl = data['predicate'][i]
    edgelabels[(v1, v2)] = e_lbl

#print(edge_labels)

pos = nx.spring_layout(G,k=0.15,iterations=20, scale=3)
nx.draw_networkx(G, pos=pos,font_size=8,node_color='pink')
nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edgelabels, font_size=7)

plt.show()

# SPARQL QUERIES

In [3]:
qres = g.query(
    """SELECT ?subject ?predicate ?object
WHERE {?subject ?predicate ?object} 

       """)

for row in qres:
    print(row)

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Pills'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/given_by'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_Armstrong'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Character'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/is_character'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_Armstrong'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Vera_Claythorne'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_profession'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/former_governess'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/maid'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_predicate'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/had_killed'))
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/John_Macarthur'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/has_lastname'), rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Macarthur'

In [4]:
qres = g.query(
    """PREFIX foaf: <http://UCLA_REU_2020.org/ATTWN/>
       SELECT ?bname
       WHERE {
          ?aname foaf:has_firstname ?bname.
       }""")

for row in qres:
    print(row)

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Lawrence'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Vera'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/John'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Anthony'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Edward_'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Thomas'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Emily'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Philip'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/William'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Ethel'),)


In [13]:
qres = g.query(
    """PREFIX foaf: <http://UCLA_REU_2020.org/ATTWN/>
       SELECT ?bname
       WHERE {
          ?aname foaf:has_profession ?bname.
       }""")

for row in qres:
    print(row)

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/doctor'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/housekeeper'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Army_General'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/butler'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/former_governess'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/veteran'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/retired_judge'),)


In [14]:
qres = g.query(
    """PREFIX foaf: <http://UCLA_REU_2020.org/ATTWN/>
       SELECT ?bname
       WHERE {
          ?aname foaf:has_lastname ?bname.
       }""")

for row in qres:
    print(row)

(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Brent'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Lombard'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Rogers'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Rogers'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Blore'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Armstrong'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Marston'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Wargrave'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Macarthur'),)
(rdflib.term.URIRef('http://UCLA_REU_2020.org/ATTWN/Claythorne'),)


In [1]:
qres = g.query(
    """PREFIX foaf: <http://UCLA_REU_2020.org/ATTWN/>
       SELECT ?Firstname
       WHERE {
          ?aname foaf:has_profession foaf:doctor.
          ?aname foaf:has_firstname ?Firstname.
       }""")

for row in qres:
    print(row)


NameError: name 'g' is not defined